# Route Donnerstag <br>
Since the travel-recommondations for Wednesday takes us outside of Vienna (including tours to Laxenburg, Baden, Heiligenkreuz etc.), <br> I will go straight to Thursday. 

In [1]:
from Classes_and_functions import Extractor, structure, matcher, Kulturwiki_comparer, Kulturwiki_indexer, Steinhausen_comparer, Steinhausen_indexer, table_comparer
import difflib as dl
import pandas as pd

from os import path
xml_path = path.relpath("../Data/pages_8_47.xml")

## I used the python-module BeautifulSoup for XML-Parsing.


from bs4 import BeautifulSoup   #import the module
with open(xml_path, encoding='utf8') as file:   #load & parse the whole text and turn it into an BeautifulSoup object for further analysis.
    contents = file.read()
    soup = BeautifulSoup(contents, 'xml')

In [2]:
Donnerstag = Extractor('Donnerstag.', 'hi', 'Freitag.', 'hi', 'placeName')

In [3]:
Donnerstag.extract_text_and_tags(soup)

In [4]:
places = Donnerstag.tag_list

In [5]:
clean_places = structure(places)
print(clean_places)
print(len(clean_places))

['kaiserlichen Burg', 'kaiserlichen Gruft', 'Kapuzinerkirche am Neumarkt', 'Klostergasse', 'Spitalplatz', 'Bürgerspital', 'Palais Lobkowitz', 'Augustinerkirche', ' Zanner', 'Josephsplatz', 'ReiterStatue ', 'Josephs des II', '. Hofbibliothek', 'Redoutensäle', 'naturhistorischen Museen', 'kaiserliche Reitschule', 'Michaelerplatze', 'Naturalienmuseum', 'Naturalienkabinette', 'Josephplatzes', 'Hofbibliothek', 'Augustinergang', 'AntikenKabinet', 'Mineralien=Kabinet', 'Palais des  Erzherzogs Karl', 'Privat=Bibliothek und Kupferstich=Sammlung', 'Blinden=Jnstitut', 'Casino', 'Schwan', 'neuen Markt', 'Stadt Frankfurt', 'Matschakerhof', 'Spiegelgasse ', 'Schönbrunn', 'Hietzing', 'Mariahilf', 'Palais Esterhazy', 'Fünfhaus', 'Braunhirschen', 'Rustendorf', 'Schönbrunn', 'Orangerie', 'Tirolerhaus', 'Menagerie', 'Pflanzengarten', 'Hietzing', 'Villa', 'Hügel', 'Dommayers Casino', 'Penzing']
50


In [6]:
import functools
print(clean_places.index('ReiterStatue '))
clean_places[10:12] = [functools.reduce(lambda x, y: x + y, clean_places[10:12])]

10


In [7]:
place_list = matcher(clean_places)
print(place_list)

['kaiserlichen Burg', 'kaiserlichen Gruft', 'Kapuzinerkirche am Neumarkt', 'Klostergasse', 'Spitalplatz', 'Bürgerspital', 'Palais Lobkowitz', 'Augustinerkirche', ' Zanner', 'Josephsplatz', 'ReiterStatue Josephs des II', '. Hofbibliothek', 'Redoutensäle', 'naturhistorischen Museen', 'kaiserliche Reitschule', 'Michaelerplatze', 'Naturalienmuseum', 'Naturalienkabinette', 'Josephplatzes', '. Hofbibliothek', 'Augustinergang', 'AntikenKabinet', 'Mineralien=Kabinet', 'Palais des  Erzherzogs Karl', 'Privat=Bibliothek und Kupferstich=Sammlung', 'Blinden=Jnstitut', 'Casino', 'Schwan', 'neuen Markt', 'Stadt Frankfurt', 'Matschakerhof', 'Spiegelgasse ', 'Schönbrunn', 'Hietzing', 'Mariahilf', 'Palais Esterhazy', 'Fünfhaus', 'Braunhirschen', 'Rustendorf', 'Schönbrunn', 'Orangerie', 'Tirolerhaus', 'Menagerie', 'Pflanzengarten', 'Hietzing', 'Villa', 'Hügel', 'Dommayers Casino', 'Penzing']


In [8]:
coordinates =  Kulturwiki_comparer(place_list)
#coordinates.head(50)

In [9]:
coordinates_choice = coordinates.drop(coordinates[coordinates['MATCH_SCORE'] <= 85].index)

In [10]:
place_coord = Kulturwiki_indexer(coordinates_choice, place_list)

In [11]:
Steinhausen_coord = Steinhausen_comparer(place_list)
#Steinhausen_coord.head(50)

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '<...>']
Applied processor reduces input query

In [12]:
Steinhausen_choice = Steinhausen_coord.drop(Steinhausen_coord[Steinhausen_coord['MATCH_SCORE'] <= 81].index)

In [13]:
place_Sth_coord = Steinhausen_indexer(Steinhausen_choice, place_list)

In [14]:
place_coord['SHAPE'] = place_coord['SHAPE'].str.replace(r'POINT', '', regex=True)
place_coord['SHAPE'] = place_coord['SHAPE'].str.replace(r'\)', '', regex=True)
place_coord[['Longitude', 'Latitude']] = place_coord['SHAPE'].str.extract('(\d+.\d+)\s(\d+.\d+)', expand=True)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\sarah\AppData\Local\Temp\ipykernel_17348\1291391956.py:3: SyntaxWarning: invalid escape sequence '\d'
  place_coord[['Longitude', 'Latitude']] = place_coord['SHAPE'].str.extract('(\d+.\d+)\s(\d+.\d+)', expand=True)


In [15]:
joined_places = pd.merge(place_coord, place_Sth_coord, left_index=True, right_index=True)
joined_places = joined_places.drop(columns=["NEUE_NAMEN_y", "SHAPE"])

In [16]:
joined_places

,NEUE_NAMEN_x,ADRESSE,Longitude_x,Latitude_x,Longitude_y,Latitude_y
0,kaiserlichen Burg,NaN,NaN,NaN,NaN,NaN
1,kaiserlichen Gruft,NaN,NaN,NaN,NaN,NaN
2,Kapuzinerkirche am Neumarkt,NaN,NaN,NaN,NaN,NaN
3,Klostergasse,"18., Klostergasse 25(SCHULE)",16.334169889219616,48.22637943373633,NaN,NaN
4,Spitalplatz,NaN,NaN,NaN,NaN,NaN
5,Bürgerspital,"01., Lobkowitzplatz 1",16.368879199407257,48.20549506608481,NaN,NaN
6,Palais Lobkowitz,NaN,NaN,NaN,NaN,NaN
7,Augustinerkirche,"01., Hofburg B Augustinerkloster",16.368033169080604,48.20518475469686,NaN,NaN
8,Zanner,NaN,NaN,NaN,NaN,NaN
9,Josephsplatz,"01., Hofburg B Augustinertrakt",16.367075752409782,48.2061750322806,NaN,NaN


In [17]:
matches_sun, matches_mon, matches_tues = table_comparer(joined_places)

In [18]:
matches_sun

,NEUE_NAMEN_x,ADRESSE_x,Longitude_x,Latitude_x,Longitude_y,Latitude_y,ADRESSE_y,Sicherheit,Longitude,Latitude,Quelle,URL
0,Palais des Erzherzogs Karl,NaN,NaN,NaN,NaN,NaN,"01., Seilerstätte 30",niedrig,16.373083,48.20375,"Wikipedia, GeoHack",https://geohack.toolforge.org/geohack.php?page...
1,Palais des Erzherzogs Karl,NaN,NaN,NaN,NaN,NaN,"01., Seilerstätte 30",niedrig,16.373083,48.20375,"Wikipedia, GeoHack",https://geohack.toolforge.org/geohack.php?page...
2,Schönbrunn,NaN,NaN,NaN,16.370375,48.21018,NaN,NaN,16.370375,48.21018,Gazetteer_Steinhausenplan_V5_auto,NaN
3,Schönbrunn,NaN,NaN,NaN,16.370375,48.21018,NaN,NaN,16.370375,48.21018,Gazetteer_Steinhausenplan_V5_auto,NaN


In [19]:
matches_mon

,NEUE_NAMEN_x,ADRESSE_x,Longitude_x_x,Latitude_x_x,Longitude_y,Latitude_y,ADRESSE_y,Sicherheit,Longitude_x_y,Latitude_x_y,Quelle,URL


In [20]:
matches_tues

,NEUE_NAMEN_x,ADRESSE_x,Longitude_x,Latitude_x,Longitude_y,Latitude_y,ADRESSE_y,Sicherheit,Longitude,Latitude,Quelle,URL


In [21]:
joined_places.to_excel('../Data/Donnerstag/joined_Donnerstag_coord.xlsx')